<a href="https://colab.research.google.com/github/zhiquanlao/Qiskit-and-Cirq-Learning-Project/blob/main/Quantum_Fourier_Transform%2C_Phase_Estimation_and_Factoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit ipywidgets

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 13.9 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 3.5 MB/s eta 0:00:00


Given a $n$-qbit unitary operation $U$ and an $n$-qbits eigenstate $|\psi\rangle$, find $\theta\in[0,1)$ s.t. $U|\psi\rangle=e^{2\pi i\theta}|\psi\rangle$

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import QFT
from qiskit.visualization import plot_histogram
import collections
import numpy as np

#Implement Quantum Fourier Transform with n qbits
def Quantum_Fourier_Transform(n, inverse):
  qc = QuantumCircuit(n)
  if inverse==False:
    theta=2*np.pi
  else:
    theta=-2*np.pi
  for i in range(n):
    qc.h(n-i-1)
    for k in range(2,n-i+1):
      qc.cp(theta/(2**k),n-i-k,n-i-1)
  for i in range(round(np.floor(n/2.0))):
    qc.swap(i,n-i-1)
  return qc


def binary_to_tenth(s):
  result=0
  for i in s.len():
    result+=int(s[s.len()-i-1])*(2**i)
  return result


#Phase estimation. For convenience we make U to be a phase gate with eigenvalue theta. n_tar=number of target of c-U gate.
#In principle, we do not know theta, but we will check the output of the function matched theta by phase estimation algo
def Phase_Estimation(theta, n_tar, n_reg,num_sample):
  def sampling(theta,n_tar,n_reg):
    label=""
    for i in range(0,n_reg):
      label=label+"+"
    for i in range(0,n_tar):
      label="1"+label
    init=Statevector.from_label(label)
    #display(init.draw("latex"))
    qc=QuantumCircuit(n_tar+n_reg)
    for i in range(n_reg):
      qc.cp(2*np.pi*theta*2**i,i,range(n_reg,n_tar+n_reg))
    qc.compose(Quantum_Fourier_Transform(n_reg,inverse=True),inplace=True)
    #display(qc.draw())
    fin=init.evolve(qc)
    #display(fin.draw("latex"))
    return fin.measure(range(0,n_reg))[0]
  output_counts = collections.defaultdict(int)
  for _ in range(num_sample):
    result=sampling(theta,n_tar,n_reg)
    output_counts[result]+=1
  max_output = max(output_counts, key=output_counts.get)
  return int(max_output,2)/(2**n_reg)



theta_ex=[0.05,0.33,0.7,0.92]
n_reg_ex=[3,5,7]
for theta in theta_ex:
  for n_reg in n_reg_ex:
    print("theta="+str(theta)+", n_reg="+str(n_reg)+", result="+str(Phase_Estimation(theta,1,n_reg,50)))


theta=0.05, n_reg=3, result=0.0
theta=0.05, n_reg=5, result=0.0625
theta=0.05, n_reg=7, result=0.046875
theta=0.33, n_reg=3, result=0.375
theta=0.33, n_reg=5, result=0.34375
theta=0.33, n_reg=7, result=0.328125
theta=0.7, n_reg=3, result=0.75
theta=0.7, n_reg=5, result=0.6875
theta=0.7, n_reg=7, result=0.703125
theta=0.92, n_reg=3, result=0.875
theta=0.92, n_reg=5, result=0.90625
theta=0.92, n_reg=7, result=0.921875


Order Finding

Given integer $N$, $a$, with $gcd(N,a)=1$, find the smallest positive integer $r$ s.t. $a^r\equiv 1\ mod(N)$